# Imports

In [1]:
import cv2
import os
import pyrebase
import pandas as pd
import numpy as np
import lungs_finder as lf

from pathlib import Path
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

from firebase_env import firebase_config

# Load the raw dataset

Loading the the read csv to control or get the directory for classes, kind and flag.

In [4]:
dataset = pd.read_csv("data_image.csv")
dataset.head()

,image_name,full_path_raw,full_path_transformed,kind,flag
0,IM-0115-0001.jpeg,raw\train\NORMAL\IM-0115-0001.jpeg,transformed\train\NORMAL\IM-0115-0001.jpeg,train,0
1,IM-0117-0001.jpeg,raw\train\NORMAL\IM-0117-0001.jpeg,transformed\train\NORMAL\IM-0117-0001.jpeg,train,0
2,IM-0119-0001.jpeg,raw\train\NORMAL\IM-0119-0001.jpeg,transformed\train\NORMAL\IM-0119-0001.jpeg,train,0
3,IM-0122-0001.jpeg,raw\train\NORMAL\IM-0122-0001.jpeg,transformed\train\NORMAL\IM-0122-0001.jpeg,train,0
4,IM-0125-0001.jpeg,raw\train\NORMAL\IM-0125-0001.jpeg,transformed\train\NORMAL\IM-0125-0001.jpeg,train,0


<IPython.core.display.Javascript object>

Defining the paths for classes...

In [5]:
raw = 'raw'
transformed = 'transformed'

path_train_normal = os.path.join("train","NORMAL")
path_train_pneu = os.path.join("train","PNEUMONIA")

path_test_normal = os.path.join("test","NORMAL")
path_test_pneu = os.path.join("test","PNEUMONIA")

path_val_normal = os.path.join("val","NORMAL")
path_val_pneu =  os.path.join("val","PNEUMONIA")

<IPython.core.display.Javascript object>

Creating the paths that will be used to save the images processed...

In [6]:
Path(os.path.join(transformed, path_train_normal)).mkdir(parents=True, exist_ok=True)
Path(os.path.join(transformed, path_train_pneu)).mkdir(parents=True, exist_ok=True)
Path(os.path.join(transformed, path_test_normal)).mkdir(parents=True, exist_ok=True)
Path(os.path.join(transformed, path_test_pneu)).mkdir(parents=True, exist_ok=True)
Path(os.path.join(transformed, path_val_normal)).mkdir(parents=True, exist_ok=True)
Path(os.path.join(transformed, path_val_pneu)).mkdir(parents=True, exist_ok=True)

<IPython.core.display.Javascript object>

In [7]:
IMAGE_RESOLUTION = (250, 250, 1)
BORDER = 30
print('Image shape: {}'.format(IMAGE_RESOLUTION))

images_transformed = Parallel(n_jobs=4)(
    delayed(process_data)(img_path, IMAGE_RESOLUTION, BORDER) for img_path in dataset.full_path_raw
)

Image shape: (250, 250, 1)
Wall time: 1min 32s


<IPython.core.display.Javascript object>

In [8]:
results = Parallel(n_jobs=4)(delayed(cv2.imwrite)(img_path_transformed, images_transformed[index]*255) for index, img_path_transformed in enumerate(dataset.full_path_transformed))

if sum(results) == len(dataset.full_path_raw):
    print("All images were transformed!")

All images were transformed!
Wall time: 13.9 s


<IPython.core.display.Javascript object>

# Firebase Connection

Saving all images on the database.

In [43]:
firebase = pyrebase.initialize_app(firebase_config)
storage = firebase.storage()

<IPython.core.display.Javascript object>

In [46]:
def save_image_firebase(image_path, storage):
    try:
        storage.child(image_path).put(image_path)
        return True
    except:
        return False


for img_path_transformed in tqdm(dataset.full_path_transformed):
    # To linux
    aux = img_path_transformed.replace("\\", "/")
    save_image_firebase(aux, storage)

<IPython.core.display.Javascript object>